In [1]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["Monitoring"]
collection = db["PRJ-16"]

In [5]:
combined_pipeline = [
    {
        '$match': {
            '$or': [
                {'T1.t': {'$exists': True}, 'A30.t': {'$exists': True}},
                {'T2.t': {'$exists': True}, 'A31.t': {'$exists': True}},
                {'T3.t': {'$exists': True}, 'A32.t': {'$exists': True}}
            ]
        }
    },
    {
        '$addFields': {
            'A30_filtered': {
                '$cond': {
                    'if': {'$gt': ['$A30.t', 60]},
                    'then': None,
                    'else': '$A30.t'
                }
            },
            'A31_filtered': {
                '$cond': {
                    'if': {'$gt': ['$A31.t', 60]},
                    'then': None,
                    'else': '$A31.t'
                }
            },
            'A32_filtered': {
                '$cond': {
                    'if': {'$gt': ['$A32.t', 60]},
                    'then': None,
                    'else': '$A32.t'
                }
            }
        }
    },
    {
        '$addFields': {
            'hour': {
                '$dateTrunc': {
                    'date': {'$toDate': '$time.datetime'},
                    'unit': 'hour'
                }
            }
        }
    },
    {
        '$group': {
            '_id': '$hour',
            'T1_values': {'$push': '$T1.t'},
            'A30_values': {'$push': '$A30_filtered'},
            'T2_values': {'$push': '$T2.t'},
            'A31_values': {'$push': '$A31_filtered'},
            'T3_values': {'$push': '$T3.t'},
            'A32_values': {'$push': '$A32_filtered'}
        }
    },
    {
        '$project': {
            'hour': '$_id',
            '_id': 0,
            'median_T1': {
                '$let': {
                    'vars': {
                        'sorted_T1': {
                            '$sortArray': {
                                'input': '$T1_values',
                                'sortBy': 1
                            }
                        }
                    },
                    'in': {
                        '$arrayElemAt': [
                            '$$sorted_T1', {
                                '$floor': {
                                    '$divide': [
                                        {'$size': '$$sorted_T1'}, 2
                                    ]
                                }
                            }
                        ]
                    }
                }
            },
            'median_A30': {
                '$cond': {
                    'if': {'$gt': [{'$size': {'$filter': {'input': '$A30_values', 'as': 'value', 'cond': {'$ne': ['$$value', None]}}}}, 0]},
                    'then': {
                        '$let': {
                            'vars': {
                                'sorted_A30': {
                                    '$sortArray': {
                                        'input': {'$filter': {'input': '$A30_values', 'as': 'value', 'cond': {'$ne': ['$$value', None]}}},
                                        'sortBy': 1
                                    }
                                }
                            },
                            'in': {
                                '$arrayElemAt': [
                                    '$$sorted_A30', {
                                        '$floor': {
                                            '$divide': [
                                                {'$size': '$$sorted_A30'}, 2
                                            ]
                                        }
                                    }
                                ]
                            }
                        }
                    },
                    'else': None
                }
            },
            'median_T2': {
                '$let': {
                    'vars': {
                        'sorted_T2': {
                            '$sortArray': {
                                'input': '$T2_values',
                                'sortBy': 1
                            }
                        }
                    },
                    'in': {
                        '$arrayElemAt': [
                            '$$sorted_T2', {
                                '$floor': {
                                    '$divide': [
                                        {'$size': '$$sorted_T2'}, 2
                                    ]
                                }
                            }
                        ]
                    }
                }
            },
            'median_A31': {
                '$cond': {
                    'if': {'$gt': [{'$size': {'$filter': {'input': '$A31_values', 'as': 'value', 'cond': {'$ne': ['$$value', None]}}}}, 0]},
                    'then': {
                        '$let': {
                            'vars': {
                                'sorted_A31': {
                                    '$sortArray': {
                                        'input': {'$filter': {'input': '$A31_values', 'as': 'value', 'cond': {'$ne': ['$$value', None]}}},
                                        'sortBy': 1
                                    }
                                }
                            },
                            'in': {
                                '$arrayElemAt': [
                                    '$$sorted_A31', {
                                        '$floor': {
                                            '$divide': [
                                                {'$size': '$$sorted_A31'}, 2
                                            ]
                                        }
                                    }
                                ]
                            }
                        }
                    },
                    'else': None
                }
            },
            'median_T3': {
                '$let': {
                    'vars': {
                        'sorted_T3': {
                            '$sortArray': {
                                'input': '$T3_values',
                                'sortBy': 1
                            }
                        }
                    },
                    'in': {
                        '$arrayElemAt': [
                            '$$sorted_T3', {
                                '$floor': {
                                    '$divide': [
                                        {'$size': '$$sorted_T3'}, 2
                                    ]
                                }
                            }
                        ]
                    }
                }
            },
            'median_A32': {
                '$cond': {
                    'if': {'$gt': [{'$size': {'$filter': {'input': '$A32_values', 'as': 'value', 'cond': {'$ne': ['$$value', None]}}}}, 0]},
                    'then': {
                        '$let': {
                            'vars': {
                                'sorted_A32': {
                                    '$sortArray': {
                                        'input': {'$filter': {'input': '$A32_values', 'as': 'value', 'cond': {'$ne': ['$$value', None]}}},
                                        'sortBy': 1
                                    }
                                }
                            },
                            'in': {
                                '$arrayElemAt': [
                                    '$$sorted_A32', {
                                        '$floor': {
                                            '$divide': [
                                                {'$size': '$$sorted_A32'}, 2
                                            ]
                                        }
                                    }
                                ]
                            }
                        }
                    },
                    'else': None
                }
            }
        }
    },
    {
        '$sort': {
            'hour': 1
        }
    }
]

In [6]:
import pandas as pd


# Convert the results to a pandas DataFrame
df = pd.DataFrame(list(collection.aggregate(combined_pipeline)))

In [7]:
df['diff_T1_A30'] = df['median_A30'] - df['median_T1']
df['diff_T2_A31'] = df['median_A31'] - df['median_T2']
df['diff_T3_A32'] = df['median_A32'] - df['median_T3']

In [8]:
df[['diff_T1_A30', 'diff_T2_A31', 'diff_T3_A32']].describe()

,diff_T1_A30,diff_T2_A31,diff_T3_A32
count,671.000000,767.000000,798.000000
mean,2.468983,0.658735,3.127193
std,0.480349,0.413670,0.594895
min,1.375000,-0.062500,1.187500
25%,2.125000,0.312500,2.812500
50%,2.375000,0.562500,3.187500
75%,2.750000,0.937500,3.437500
max,3.812500,2.062500,4.937500


In [9]:
import plotly.graph_objs as go

def plot_median_series(df, series1, series2, title='Temperatures of TLeaf and ALeaf'):
    fig = go.Figure()

    # Add the first series
    fig.add_trace(go.Scatter(
        x=df['hour'], y=df[series1], mode='lines', name=f'Median {series1.split("_")[1]}'))

    # Add the second series
    fig.add_trace(go.Scatter(
        x=df['hour'], y=df[series2], mode='lines', name=f'Median {series2.split("_")[1]}'))

    # Update layout
    fig.update_layout(title=title,
                      xaxis_title='Hour',
                      yaxis_title='Median Values',
                      legend_title='Series',
                      legend=dict(
                          orientation="h",
                          yanchor="bottom",
                          y=1.02,
                          xanchor="left",
                          x=0.5
                      ),
                      hovermode="x unified"
                      )

    # Show the figure
    fig.show()

In [10]:
# Example usage
plot_median_series(df, 'median_T1', 'median_A30')

In [11]:
# Example usage
plot_median_series(df, 'median_T2', 'median_A31')

In [12]:
# Example usage
plot_median_series(df, 'median_T3', 'median_A32')

In [13]:
import plotly.graph_objects as go

# Create traces for the differences
trace_diff_1 = go.Scatter(
    x=df['hour'],
    y=df['diff_T1_A30'],
    mode='lines',
    name='Diff T1-A30'
)

trace_diff_2 = go.Scatter(
    x=df['hour'],
    y=df['diff_T2_A31'],
    mode='lines',
    name='Diff T2-A31'
)

trace_diff_3 = go.Scatter(
    x=df['hour'],
    y=df['diff_T3_A32'],
    mode='lines',
    name='Diff T3-A32'
)

# Create a figure and add traces
fig = go.Figure()

fig.add_trace(trace_diff_1)
fig.add_trace(trace_diff_2)
fig.add_trace(trace_diff_3)

# Update the layout with titles and legend positioning
fig.update_layout(
    title='Temperature Differences Between TLeaf and ALeaf Over Time',
    xaxis_title='Hour',
    yaxis_title='Difference',
    legend_title='Series',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    hovermode="x unified"
)

# Show the figure
fig.show()